In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#也可对每张表的列去重
# 每个文件先导进来
# 同一张表先拼接上
# 每种表有什么特征
# 表互相怎么连接
# 把表有必要的部分连接到base表上
# 然后EDA 分析
# 除了主表外，还有9张数据表+两张说明表

# applprev,
# credit_bureau,
# debitcard,
# deposit,
# other,
# person,
# static,
# static_cb_0,
# tax_registry

In [3]:
# 导入机器学习常用的包
import os
import gc
from glob import glob
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.base import BaseEstimator, ClassifierMixin

import lightgbm as lgb

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

ROOT = Path("/kaggle/input/home-credit-credit-risk-model-stability")
TRAIN_DIR = ROOT / "parquet_files" / "train"
TEST_DIR = ROOT / "parquet_files" / "test"

In [4]:
def read_file(path, depth=None):
    df = pl.read_parquet(path)
    return process_dataframe(df, depth)

def read_files(regex_path, depth=None):
    import glob as glob
    # 使用列表推导和上下文管理器读取并处理所有文件
    chunks = [process_dataframe(pl.read_parquet(path), depth) for path in glob.glob(str(regex_path))]
    
    # 合并所有 DataFrame 并去重
    df = pl.concat(chunks, how="vertical_relaxed")
    df = df.unique(subset=["case_id"])
    
    return df



In [5]:
def __clear(data_store):
    del data_store
    gc.collect()

In [6]:
def to_pandas(df_data, cat_cols=None):
    df_data = df_data.to_pandas()
    
    if cat_cols is None:
        cat_cols = list(df_data.select_dtypes("object").columns)
    
    df_data[cat_cols] = df_data[cat_cols].astype("category")
    
    return df_data, cat_cols

In [7]:
class VotingModel(BaseEstimator, ClassifierMixin):
    def __init__(self, estimators):
        super().__init__()
        self.estimators = estimators
        
    def fit(self, X, y=None):
        return self
    
    def predict(self, X):
        y_preds = [estimator.predict(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)
    
    def predict_proba(self, X):
        y_preds = [estimator.predict_proba(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)

In [8]:
class Pipeline:
    @staticmethod
    def set_table_dtypes(df):
        # 定义列名到数据类型的映射
        dtype_mapping = {
            "case_id": pl.Int32,
            "WEEK_NUM": pl.Int32,
            "num_group1": pl.Int32,
            "num_group2": pl.Int32,
            "date_decision": pl.Date,
        }

        # 根据列名的最后一个字符定义数据类型
        for col in df.columns:
            if col in dtype_mapping:
                df = df.with_columns(pl.col(col).cast(dtype_mapping[col]))
            elif col[-1] in ("P", "A"):
                df = df.with_columns(pl.col(col).cast(pl.Float64))
            elif col[-1] == "M":
                df = df.with_columns(pl.col(col).cast(pl.String))
            elif col[-1] == "D":
                df = df.with_columns(pl.col(col).cast(pl.Date))

        return df

    @staticmethod
    def handle_dates(df):
        # 处理以 "D" 结尾的日期列
        date_cols = [col for col in df.columns if col[-1] == "D"]
        for col in date_cols:
            df = df.with_columns(
                (pl.col(col) - pl.col("date_decision")).alias(col)
            ).with_columns(
                pl.col(col).dt.total_days().cast(pl.Float32).alias(col)
            )

        # 删除不需要的列
        df = df.drop(["date_decision", "MONTH"])

        return df

    @staticmethod
    def filter_cols(df):
        # 过滤掉缺失值比例超过 95% 的列
        null_ratio = df.select([pl.col(col).is_null().mean().alias(col) for col in df.columns])
        high_null_cols = [col for col in null_ratio.columns if null_ratio[col][0] > 0.95 and col not in ["target", "case_id", "WEEK_NUM"]]
        df = df.drop(high_null_cols)

        # 过滤掉唯一值数量为 1 或超过 200 的字符串列
        string_cols = [col for col in df.columns if df[col].dtype == pl.String]
        for col in string_cols:
            if col not in ["target", "case_id", "WEEK_NUM"]:
                freq = df[col].n_unique()
                if freq == 1 or freq > 200:
                    df = df.drop(col)

        return df

def process_dataframe(df, depth=None):
    df = Pipeline.set_table_dtypes(df)
    
    if depth in [1, 2]:
        df = df.group_by("case_id").agg(Aggregator.get_exprs(df))
    
    return df

In [9]:
class Aggregator:
    @staticmethod
    def num_expr(df):
        cols = [col for col in df.columns if col[-1] in ("P", "A")]

        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]

        return expr_max

    @staticmethod
    def date_expr(df):
        cols = [col for col in df.columns if col[-1] in ("D",)]

        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]

        return expr_max

    @staticmethod
    def str_expr(df):
        cols = [col for col in df.columns if col[-1] in ("M",)]
        
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]

        return expr_max

    @staticmethod
    def other_expr(df):
        cols = [col for col in df.columns if col[-1] in ("T", "L")]
        
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]

        return expr_max
    
    @staticmethod
    def count_expr(df):
        cols = [col for col in df.columns if "num_group" in col]

        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]

        return expr_max

    @staticmethod
    def get_exprs(df):
        exprs = Aggregator.num_expr(df) + \
                Aggregator.date_expr(df) + \
                Aggregator.str_expr(df) + \
                Aggregator.other_expr(df) + \
                Aggregator.count_expr(df)

        return exprs

In [10]:
data_store = {
    "df_base": read_file(TRAIN_DIR / "train_base.parquet"),
    "depth_0": [
        read_file(TRAIN_DIR / "train_static_cb_0.parquet"),
        read_files(TRAIN_DIR / "train_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TRAIN_DIR / "train_applprev_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_a_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_c_1.parquet", 1),
        read_files(TRAIN_DIR / "train_credit_bureau_a_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_credit_bureau_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_other_1.parquet", 1),
        read_file(TRAIN_DIR / "train_person_1.parquet", 1),
        read_file(TRAIN_DIR / "train_deposit_1.parquet", 1),
        read_file(TRAIN_DIR / "train_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TRAIN_DIR / "train_credit_bureau_b_2.parquet", 2),
        read_files(TRAIN_DIR / "train_credit_bureau_a_2_*.parquet", 2),
    ]
}

FileNotFoundError: 系统找不到指定的路径。 (os error 3): ...gle\input\home-credit-credit-risk-model-stability\parquet_files\train\train_base.parquet

In [11]:
def feature_eng(df_base, depth_0, depth_1, depth_2):
    df_base = (
        df_base
        .with_columns(
            month_decision = pl.col("date_decision").dt.month(),
            weekday_decision = pl.col("date_decision").dt.weekday(),
        )
    )
        
    for i, df in enumerate(depth_0 + depth_1 + depth_2):
        df_base = df_base.join(df, how="left", on="case_id", suffix=f"_{i}")
        
    df_base = df_base.pipe(Pipeline.handle_dates)
    
    return df_base

In [12]:
df_train = feature_eng(**data_store)

print("train data shape:\t", df_train.shape)

train data shape:	 (1526659, 472)


In [13]:
data_store = {
    "df_base": read_file(TEST_DIR / "test_base.parquet"),
    "depth_0": [
        read_file(TEST_DIR / "test_static_cb_0.parquet"),
        read_files(TEST_DIR / "test_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TEST_DIR / "test_applprev_1_*.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_a_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_b_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_c_1.parquet", 1),
        read_files(TEST_DIR / "test_credit_bureau_a_1_*.parquet", 1),
        read_file(TEST_DIR / "test_credit_bureau_b_1.parquet", 1),
        read_file(TEST_DIR / "test_other_1.parquet", 1),
        read_file(TEST_DIR / "test_person_1.parquet", 1),
        read_file(TEST_DIR / "test_deposit_1.parquet", 1),
        read_file(TEST_DIR / "test_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TEST_DIR / "test_credit_bureau_b_2.parquet", 2),
        read_files(TEST_DIR / "test_credit_bureau_a_2_*.parquet", 2),
    ]
}

In [14]:
df_test = feature_eng(**data_store)

print("test data shape:\t", df_test.shape)

test data shape:	 (10, 471)


In [15]:
df_train = df_train.pipe(Pipeline.filter_cols)
df_test = df_test.select([col for col in df_train.columns if col != "target"])

print("train data shape:\t", df_train.shape)
print("test data shape:\t", df_test.shape)

train data shape:	 (1526659, 361)
test data shape:	 (10, 360)


In [16]:
df_train, cat_cols = to_pandas(df_train)
df_test, cat_cols = to_pandas(df_test, cat_cols)

In [17]:
__clear(data_store)

In [18]:
X = df_train.drop(columns=["target", "case_id", "WEEK_NUM"])
y = df_train["target"]
weeks = df_train["WEEK_NUM"]

cv = StratifiedGroupKFold(n_splits=5, shuffle=False)

params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 8,
    "learning_rate": 0.05,
    "n_estimators": 1500,
    "colsample_bytree": 0.8, 
    "colsample_bynode": 0.8,
    "verbose": -1,
    "random_state": 42,
    "device": "gpu",
}

fitted_models = []

for idx_train, idx_valid in cv.split(X, y, groups=weeks):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]

    model = lgb.LGBMClassifier(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[lgb.log_evaluation(100), lgb.early_stopping(100)]
    )

    fitted_models.append(model)

model = VotingModel(fitted_models)

1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.837898
[200]	valid_0's auc: 0.84731
[300]	valid_0's auc: 0.850355
[400]	valid_0's auc: 0.851604
[500]	valid_0's auc: 0.852133
[600]	valid_0's auc: 0.852623
[700]	valid_0's auc: 0.853076
[800]	valid_0's auc: 0.853342
[900]	valid_0's auc: 0.853546
[1000]	valid_0's auc: 0.853647
[1100]	valid_0's auc: 0.85385
[1200]	valid_0's auc: 0.854002
[1300]	valid_0's auc: 0.854132
[1400]	valid_0's auc: 0.85415
Early stopping, best iteration is:
[1341]	valid_0's auc: 0.854188
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.838392
[200]	valid_0's auc: 0.847511
[300]	valid_0's auc: 0.85053
[400]	valid_0's auc: 0.851656
[500]	valid_0's auc: 0.852333
[600]	valid_0's auc: 0.852704
[700]	valid_0's auc: 0.853051
[800]	valid_0's auc: 0.853222
[900]	valid_0's auc: 0.85332
[1000]	valid_0's auc: 0.853394
[1100]	valid_0's auc: 0.853422
[1200]	valid_0's auc: 0.853477
Early stopping, best iterat

In [19]:
X_test = df_test.drop(columns=["WEEK_NUM"])
X_test = X_test.set_index("case_id")

y_pred = pd.Series(model.predict_proba(X_test)[:, 1], index=X_test.index)

In [20]:
df_subm = pd.read_csv(ROOT / "sample_submission.csv")
df_subm = df_subm.set_index("case_id")

df_subm["score"] = y_pred

In [21]:
print("Check null: ", df_subm["score"].isnull().any())

print(df_subm.head())
df_subm.to_csv("submission.csv")

Check null:  False
            score
case_id          
57543    0.004910
57549    0.070310
57551    0.004767
57552    0.015138
57569    0.128849


In [22]:
from sklearn.metrics import roc_auc_score
def gini_stability(base, w_fallingrate=88.0, w_resstd=-0.5):
    gini_in_time = base.loc[:, ["WEEK_NUM", "target", "score"]]\
        .sort_values("WEEK_NUM")\
        .groupby("WEEK_NUM")[["target", "score"]]\
        .apply(lambda x: 2*roc_auc_score(x["target"], x["score"])-1).tolist()
    
    x = np.arange(len(gini_in_time))
    y = gini_in_time
    a, b = np.polyfit(x, y, 1)
    y_hat = a*x + b
    residuals = y - y_hat
    res_std = np.std(residuals)
    avg_gini = np.mean(gini_in_time)
    return avg_gini + w_fallingrate * min(0, a) + w_resstd * res_std

In [23]:
# gini_stability()

In [24]:

# 预测
y_pred = model.predict(X_test)

# 评估模型
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')
print(classification_report(y_test, y_pred))

# 特征重要性
feature_importances = model.get_feature_importance(Pool(X_test, label=y_test))
feature_names = X_test.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print(f'{name}: {score}')

# 保存模型
model.save_model('catboost_gpu_model.cbm')

NameError: name 'accuracy_score' is not defined

In [ ]:
# from catboost import CatBoostClassifier, Pool

# X = df_train.drop(columns=["target", "case_id", "WEEK_NUM"])
# y = df_train["target"]
# weeks = df_train["WEEK_NUM"]

# cv = StratifiedGroupKFold(n_splits=5, shuffle=False)

# cat_features_list = [col for col in X.columns if X[col].dtype.name in ['object', 'category']]
# print(cat_features)
# X[cat_features_list] = X[cat_features_list].astype(str)

# fitted_models_cb = []

# for idx_train, idx_valid in cv.split(X, y, groups=weeks):
#     X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
#     X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]

#     model = CatBoostClassifier(
#         iterations=1000,  # 迭代次数
#         learning_rate=0.03,  # 学习率
#         depth=6,  # 树的深度
#         loss_function='Logloss',  # 损失函数
#         eval_metric='Accuracy',  # 评估指标
#         random_seed=42,  # 随机种子
#         verbose=100,  # 每 100 次迭代打印一次信息
#         task_type='GPU',  # 使用 GPU
#         devices='0:1'  # 使用哪些 GPU 设备，例如 '0:1' 表示使用第 0 和第 1 号 GPU
#     )

#     # 训练模型
#     model.fit(
#         X_train, y_train,
#         eval_set=[(X_valid, y_valid)],
#         early_stopping_rounds=500,  # 早停，如果 100 轮内没有提升则停止训练
#         use_best_model=True,  # 使用最佳模型
#         cat_features=cat_features_list,
#         plot=True  # 绘制训练过程中的学习曲线
#     )

#     fitted_models_cb.append(model)

# model = VotingModel(fitted_models_cb)